In [35]:
# Import the correct algorithm
from surprise import Dataset, Reader, accuracy, SVD, CoClustering, SlopeOne 
from surprise.accuracy import rmse
from surprise.model_selection import GridSearchCV, train_test_split
import pandas as pd
import time
import pathlib
import numpy as np

## Load Datasets

In [43]:
data_100k =  Dataset.load_builtin('ml-100k')
# data_1m = Dataset.load_builtin('ml-1m')
# path = pathlib.Path('ml-10M100K').resolve() / 'ratings.dat'
# reader = Reader(line_format="user item rating timestamp",sep='::')
# print(path)
# data_10m = Dataset.load_from_file(file_path=path,reader=reader)

/Users/kohjunkai/Documents/GitHub/CZ4032/ml-10M100K/ratings.dat


In [44]:
# 'user item rating timestamp', separated by '::' characters.
# reader = Reader(line_format="user item rating timestamp",sep='::')

# param_grid = {"n_epochs": [10], 
#               "lr_all": [0.002,0.003,0.005], 
#               "reg_all": [0.01,0.02,0.03],
#               'biased':[True,False],'random_state':[1],
#               'verbose':[True]}

# param_grid = {"n_cltr_u": [2,3,4], 
#               "n_cltr_i": [2,3,4], 
#               "random_state":[1],
#               "verbose":[True]}

# gs_SVD = GridSearchCV(SVD, param_grid, measures=["rmse"], cv=3, n_jobs=-1)
# gs_SlopeOne = GridSearchCV(SlopeOne, param_grid, measures=["rmse"], cv=3, n_jobs=-1)
# gs_CoClustering = GridSearchCV(CoClustering, param_grid, measures=["rmse"], cv=3, n_jobs=-1)
# gs_CoClustering.param_combinations

# gs_SVD.fit(data_100k)
# gs_SlopeOne.fit(data_100k)
# gs_CoClustering.fit(data_100k)
# print("SVD - 100k")
# print('RMSE_best_score:',gs_SVD.best_score["rmse"])
# print('RMSE_best_params',gs_SVD.best_params["rmse"])
# train_data_100k, test_data_100k = train_test_split(data_100k, test_size=0.25, random_state=1)
# algo = gs.best_estimator["rmse"]
# algo.fit(train_data_100k)

## Initialise diff algo

In [45]:
svd = SVD()
co_clustering = CoClustering()
slope_one = SlopeOne()
algo_dict = {"SVD":svd,"CoClustering":co_clustering,"SlopeOne":slope_one}

## Split training and test data

In [46]:
# Test set is 25%
train_data_100k, test_data_100k = train_test_split(data_100k, test_size=0.25, random_state=1)
# train_data_1m, test_data_1m = train_test_split(data_1m, test_size=0.25, random_state=1)
# train_data_10m, test_data_10m = train_test_split(data_10m, test_size=0.25, random_state=1)

## Compare fit timing and prediction accuracy

In [47]:
predictions = {}

print("----------- 100k ---------------")
for name,algo in algo_dict.items():
    begin = time.time()
    algo.fit(train_data_100k)
    end = time.time()
    predictions[name] = algo.test(test_data_100k)
    result = accuracy.rmse(predictions[name])
    print(f'{name}: {result}, fit_time: {round(end-begin,2)}')
    print()

# print("----------- 1m ---------------")
# for name,algo in algo_dict.items():
#     begin = time.time()
#     algo.fit(train_data_1m)
#     end = time.time()
#     predictions[name] = algo.test(test_data_1m)
#     result = accuracy.rmse(predictions[name])
#     print(f'{name}: {result}, fit_time: {round(end-begin,2)}')
#     print()

# print("----------- 10m ---------------")
# for name,algo in algo_dict.items():
#     begin = time.time()
#     algo.fit(train_data_10m)
#     end = time.time()
#     predictions[name] = algo.test(test_data_10m)
#     result = accuracy.rmse(predictions[name])
#     print(f'{name}: {result}, fit_time: {round(end-begin,2)}')
#     print()

----------- 100k ---------------
RMSE: 0.9437
SVD: 0.943690339230827, fit_time: 0.72

RMSE: 0.9685
CoClustering: 0.9685200124337181, fit_time: 1.34

RMSE: 0.9477
SlopeOne: 0.9477498649038465, fit_time: 0.48



## Preparing dataframe for predictions

In [48]:
test_data_100k = pd.DataFrame(test_data_100k)
test_data_100k = test_data_100k.rename(columns={
    0: "uid",
    1: "iid",
    2: "rating"
})

# test_data_1m = pd.DataFrame(test_data_1m)
# test_data_1m = test_data_1m.rename(columns={
#     0: "uid",
#     1: "iid",
#     2: "rating"
# })

# test_data_10m = pd.DataFrame(test_data_10m)
# test_data_10m = test_data_10m.rename(columns={
#     0: "uid",
#     1: "iid",
#     2: "rating"
# })

## Compare time taken to predict missing data

In [51]:
pred_dict_100k={}
pred_dict_1m={}
pred_dict_10m={}


print("----------- 100k ---------------")
for name,algo in algo_dict.items():
    pred_dict_100k[name] = []
    total_time = 0.00
    for i in range(len(test_data_100k)):
        begin = time.time()
        y_pred = algo.predict(test_data_100k.loc[i]['uid'], test_data_100k.loc[i]['iid'])
        end = time.time()
        y_pred = np.array(y_pred)
        y_pred[2] = test_data_100k.loc[i]['rating']
        pred_dict_100k[name].append(y_pred)
        total_time += end-begin
    print(name)
    print(f"time taken to generate test predictions = {round(total_time, 2)} seconds")
    print()
    
# print("----------- 1m ---------------")
# for name,algo in algo_dict.items():
#     pred_dict_1m[name] = []
#     total_time = 0.00
#     for i in range(len(test_data_1m)):
#         begin = time.time()
#         y_pred = algo.predict(test_data_1m.loc[i]['uid'], test_data_1m.loc[i]['iid'])
#         end = time.time()
#         y_pred = np.array(y_pred)
#         y_pred[2] = test_data_1m.loc[i]['rating']
#         pred_dict_1m[name].append(y_pred)
#         total_time += end-begin
#     print(name)
#     print(f"time taken to generate test predictions = {round(total_time, 2)} seconds")
#     print()

    
# print("----------- 10m ---------------")
# for name,algo in algo_dict.items():
#     pred_dict_10m[name] = []
#     total_time = 0.00
#     for i in range(len(test_data_10m)):
#         begin = time.time()
#         y_pred = algo.predict(test_data_10m.loc[i]['uid'], test_data_10m.loc[i]['iid'])
#         end = time.time()
#         y_pred = np.array(y_pred)
#         y_pred[2] = test_data_10m.loc[i]['rating']
#         pred_dict_10m[name].append(y_pred)
#         total_time += end-begin
#     print(name)
#     print(f"time taken to generate test predictions = {round(total_time, 2)} seconds")
#     print()


----------- 100k ---------------
SVD
time taken to generate test predictions = 3.57 seconds

CoClustering
time taken to generate test predictions = 3.43 seconds

SlopeOne
time taken to generate test predictions = 6.24 seconds



## Analysis of prediction results

In [52]:
def get_Iu(uid):
    """ return the number of items rated by given user
    args: 
      uid: the id of the user
    returns: 
      the number of items rated by the user
    """
    try:
        return len(trainset.ur[trainset.to_inner_uid(uid)])
    except ValueError: # user was not part of the trainset
        return 0
    
def get_Ui(iid):
    """ return number of users that have rated given item
    args:
      iid: the raw id of the item
    returns:
      the number of users that have rated the item.
    """
    try: 
        return len(trainset.ir[trainset.to_inner_iid(iid)])
    except ValueError:
        return 0

df_dict_100k = {}
for name,array in pred_dict_100k.items():
    trainset = algo_dict[name].trainset
    df_dict_100k[name] = pd.DataFrame(pred_dict_100k[name])
    df_dict_100k[name] = df_dict_100k[name].rename(columns={
        0: "uid",
        1: "iid",
        2: "actual rating",
        3: "predicted rating",
        4: "-"
    })
    df_dict_100k[name]['no._items_rated_by_user'] = df_dict_100k[name].uid.apply(get_Iu)
    df_dict_100k[name]['no._user_that_rated_item'] = df_dict_100k[name].iid.apply(get_Ui)
    df_dict_100k[name]['error'] = abs(df_dict_100k[name]["actual rating"]-df_dict_100k[name]["predicted rating"])

# df_dict_1m = {}
# for name,array in pred_dict_1m.items():
#     trainset = algo_dict[name].trainset
#     df_dict_1m[name] = pd.DataFrame(pred_dict_1m[name])
#     df_dict_1m[name] = df_dict_1m[name].rename(columns={
#         0: "uid",
#         1: "iid",
#         2: "actual rating",
#         3: "predicted rating",
#         4: "-"
#     })
#     df_dict_1m[name]['no._items_rated_by_user'] = df_dict_1m[name].uid.apply(get_Iu)
#     df_dict_1m[name]['no._user_that_rated_item'] = df_dict_1m[name].iid.apply(get_Ui)
#     df_dict_1m[name]['error'] = abs(df_dict_1m[name]["actual rating"]-df_dict_1m[name]["predicted rating"])

# df_dict_10m = {}
# for name,array in pred_dict_10m.items():
#     trainset = algo_dict[name].trainset
#     df_dict_10m[name] = pd.DataFrame(pred_dict_10m[name])
#     df_dict_10m[name] = df_dict_10m[name].rename(columns={
#         0: "uid",
#         1: "iid",
#         2: "actual rating",
#         3: "predicted rating",
#         4: "-"
#     })
#     df_dict_10m[name]['no._items_rated_by_user'] = df_dict_10m[name].uid.apply(get_Iu)
#     df_dict_10m[name]['no._user_that_rated_item'] = df_dict_10m[name].iid.apply(get_Ui)
#     df_dict_10m[name]['error'] = abs(df_dict_10m[name]["actual rating"]-df_dict_10m[name]["predicted rating"])


In [55]:
best_predictions = df_dict_100k["SVD"].sort_values(by='actual rating',ascending=False)[:50]
worst_predictions = df_dict_100k["SVD"].sort_values(by='actual rating',ascending=False)[-50:]

In [56]:
best_predictions

,uid,iid,actual rating,predicted rating,-,no._items_rated_by_user,no._user_that_rated_item,error
10261,664,480,5.0,4.160832,{'was_impossible': False},121,130,0.839168
18840,274,100,5.0,4.748635,{'was_impossible': False},51,377,0.251365
18830,848,428,5.0,4.244477,{'was_impossible': False},104,89,0.755523
4274,94,464,5.0,3.599637,{'was_impossible': False},291,18,1.400363
8126,94,959,5.0,3.914888,{'was_impossible': False},291,50,1.085112
4271,343,176,5.0,4.288887,{'was_impossible': False},187,222,0.711113
4268,389,494,5.0,3.983391,{'was_impossible': False},211,38,1.016609
8130,881,663,5.0,3.655222,{'was_impossible': False},208,84,1.344778
13231,330,204,5.0,4.667251,{'was_impossible': False},117,255,0.332749
4265,452,269,5.0,3.888500,{'was_impossible': False},148,233,1.111500


In [19]:
worst_predictions

,uid,iid,actual rating,predicted rating,-,no._items_rated_by_user,no._user_that_rated_item,error
24364,588,98,1.0,4.465937,{'was_impossible': False},161,282,3.465937
3224,405,1053,5.0,1.478848,{'was_impossible': False},567,18,3.521152
2535,405,47,5.0,1.450300,{'was_impossible': False},567,105,3.549700
4006,295,96,1.0,4.554957,{'was_impossible': False},139,241,3.554957
12999,777,127,1.0,4.599864,{'was_impossible': False},30,316,3.599864
557,286,285,1.0,4.672327,{'was_impossible': False},210,131,3.672327
14412,495,419,1.0,4.690803,{'was_impossible': False},171,142,3.690803
14320,382,50,1.0,4.696609,{'was_impossible': False},49,440,3.696609
8296,551,423,1.0,4.818668,{'was_impossible': False},262,239,3.818668
1990,405,571,5.0,1.000000,{'was_impossible': False},567,17,4.000000


## Analysis of worst prediction

In [ ]:
df_100k = pd.DataFrame(data_100k.__dict__['raw_ratings'], columns=['uid','iid','rating','timestamp'])

df_100k.loc[df_100k['iid'] == '1090']['rating'].describe()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib notebook

df_100k.loc[df_100k['iid'] == '1090']['rating'].hist()
plt.xlabel('rating')
plt.ylabel('Number of ratings')
plt.title('Number of ratings item 1090 has received')
plt.show();